In [31]:
module ResetMclWorld

using Plots
using Distributions
using StatsBase
using LinearAlgebra
using Printf
using Logging
Logging.disable_logging(Logging.Info)

include("../scripts/mcl.jl")
import .MclWorld: RealRobot, RealCamera
import .MclWorld: IdealAgent, Agent, EstimationAgent, TrueLandmark, Landmark, Map, Camera, Robot, World
import .MclWorld: data, decision, state_transition, circle_shape, draw, append
import .MclWorld: Estimator, Particle, set_ml, motion_update, observation_update, resampling_system

mutable struct ResetMcl <: Estimator
    init_pose::Array{Float64}
    num::Int64
    map::Map
    particles::Array{Particle}
    distance_dev_rate::Float64
    direction_dev::Float64
    motion_noise_rate_pdf::MvNormal
    ml::Particle
    pose::Array{Float64}
    alphas::Dict

    function ResetMcl(
        init_pose,
        num;
        map=Map(),
        particles=[Particle(init_pose, 1.0/num) for i in 1:num],
        distance_dev_rate=0.14,
        direction_dev=0.05,
        motion_noise_stds=Dict([("nn", 0.19), ("no", 0.001), ("on", 0.13), ("oo", 0.2)]),
        ml = particles[1],
        pose = ml.pose,
        alphas=Dict()
        )
    
        c = diagm([motion_noise_stds["nn"]^2, motion_noise_stds["no"]^2, motion_noise_stds["on"]^2, motion_noise_stds["oo"]^2])
        motion_noise_rate_pdf = MvNormal([0, 0, 0, 0], c)
    
        new(
            init_pose,
            num,
            map,
            particles,
            distance_dev_rate,
            direction_dev,
            motion_noise_rate_pdf,
            ml,
            pose,
            alphas
        )
    end

end

function observation_update(self::Estimator, observation)
    for p in self.particles
        observation_update(p, observation, self.map, self.distance_dev_rate, self.direction_dev)
    end
    
    #alpha値の記録
    alpha = sum([p.weight for p in self.particles])
    obsnum = length(observation)
    if !haskey(self.alphas, obsnum)
        self.alphas[obsnum] = []
    end
    push!(self.alphas[obsnum], alpha)
    
    set_ml(self)
    resampling_system(self)
end

end

Main.ResetMclWorld

In [32]:
using .ResetMclWorld
using Distributions

function trial(animation=true)

    time_interval = 0.1    
    world = ResetMclWorld.World(30, 0.1)

    ### 地図を生成して2つランドマークを追加 ###
    m = ResetMclWorld.Map()
    ResetMclWorld.append(m, ResetMclWorld.TrueLandmark([-4, 2]))
    ResetMclWorld.append(m, ResetMclWorld.TrueLandmark([2, -3]))
    ResetMclWorld.append(m, ResetMclWorld.TrueLandmark([3, 3]))
    ResetMclWorld.append(world, m)

    initial_pose = [0.0, 0.0, 0.0]
    pf = ResetMclWorld.ResetMcl(initial_pose, 100, map=m)
    a = ResetMclWorld.EstimationAgent(time_interval, 0.2, 10.0/180.0*pi, pf)
    r = ResetMclWorld.RealRobot(initial_pose, :red, 0.2, a, sensor=ResetMclWorld.RealCamera(m))
    ResetMclWorld.append(world, r)

    if animation
        ResetMclWorld.draw(world)
    else
        ResetMclWorld.draw(world)
        return pf
    end
end

pf = trial(false)

Main.ResetMclWorld.ResetMcl([0.0, 0.0, 0.0], 100, Main.ResetMclWorld.MclWorld.RealRobotWorld.RobotWorld.Map(Main.ResetMclWorld.MclWorld.RealRobotWorld.RobotWorld.Landmark[Main.ResetMclWorld.MclWorld.RealRobotWorld.RobotWorld.Landmark([-4.0, 2.0], 1), Main.ResetMclWorld.MclWorld.RealRobotWorld.RobotWorld.Landmark([2.0, -3.0], 2), Main.ResetMclWorld.MclWorld.RealRobotWorld.RobotWorld.Landmark([3.0, 3.0], 3)]), Main.ResetMclWorld.MclWorld.Particle[Main.ResetMclWorld.MclWorld.Particle([-1.188956285263327, 2.313741464276541, 3.990050815361451], 0.01), Main.ResetMclWorld.MclWorld.Particle([-1.188956285263327, 2.313741464276541, 3.990050815361451], 0.01), Main.ResetMclWorld.MclWorld.Particle([-1.1890346697742922, 2.311776937447829, 4.04202705292135], 0.01), Main.ResetMclWorld.MclWorld.Particle([-1.1774326967239857, 2.3254875391667893, 3.998311876206066], 0.01), Main.ResetMclWorld.MclWorld.Particle([-1.1398317785263528, 2.368015519628099, 4.038352067846699], 0.01), Main.ResetMclWorld.MclWorld.

In [33]:
for (key, val) in pf.alphas ###mclalpharesult
    println("landmarks: ", key, " particles: ", length(pf.particles), " min: ", minimum(pf.alphas[key]), " max: ", maximum(pf.alphas[key]))
end

landmarks: 0 particles: 100 min: 1.0 max: 1.0
landmarks: 2 particles: 100 min: 1.6245328717616363 max: 7.468356397769986
landmarks: 1 particles: 100 min: 0.08265269228456085 max: 8.39412140543331


In [35]:
using .ResetMclWorld
using Distributions

function trial(animation=true)

    time_interval = 0.1    
    world = ResetMclWorld.World(30, 0.1)

    ### 地図を生成して2つランドマークを追加 ###
    m = ResetMclWorld.Map()
    ResetMclWorld.append(m, ResetMclWorld.TrueLandmark([-4, 2]))
    ResetMclWorld.append(m, ResetMclWorld.TrueLandmark([2, -3]))
    ResetMclWorld.append(m, ResetMclWorld.TrueLandmark([3, 3]))
    ResetMclWorld.append(world, m)

    initial_pose = [-4.0, -4.0, 0.0]
    pf = ResetMclWorld.ResetMcl(initial_pose, 100, map=m)
    a = ResetMclWorld.EstimationAgent(time_interval, 0.2, 10.0/180.0*pi, pf)
    r = ResetMclWorld.RealRobot(initial_pose, :red, 0.2, a, sensor=ResetMclWorld.RealCamera(m))
    ResetMclWorld.append(world, r)

    if animation
        ResetMclWorld.draw(world)
    else
        ResetMclWorld.draw(world)
        return pf
    end
end

pf = trial(false)

Main.ResetMclWorld.ResetMcl([-4.0, -4.0, 0.0], 100, Main.ResetMclWorld.MclWorld.RealRobotWorld.RobotWorld.Map(Main.ResetMclWorld.MclWorld.RealRobotWorld.RobotWorld.Landmark[Main.ResetMclWorld.MclWorld.RealRobotWorld.RobotWorld.Landmark([-4.0, 2.0], 1), Main.ResetMclWorld.MclWorld.RealRobotWorld.RobotWorld.Landmark([2.0, -3.0], 2), Main.ResetMclWorld.MclWorld.RealRobotWorld.RobotWorld.Landmark([3.0, 3.0], 3)]), Main.ResetMclWorld.MclWorld.Particle[Main.ResetMclWorld.MclWorld.Particle([-4.712475084235242, -3.2873398435359986, 5.210032470352772], 0.01), Main.ResetMclWorld.MclWorld.Particle([-4.947611408045768, -2.536298959420777, 5.174157161441324], 0.01), Main.ResetMclWorld.MclWorld.Particle([-4.039925932505328, -2.9660884978669446, 5.5304310544498465], 0.01), Main.ResetMclWorld.MclWorld.Particle([-5.134589668932652, -2.609448323120949, 4.496521208525135], 0.01), Main.ResetMclWorld.MclWorld.Particle([-4.2022131319981355, -2.6334428100888765, 5.703802671942405], 0.01), Main.ResetMclWorld.

In [36]:
for (key, val) in pf.alphas ###mclalpharesult
    println("landmarks: ", key, " particles: ", length(pf.particles), " min: ", minimum(pf.alphas[key]), " max: ", maximum(pf.alphas[key]))
end

landmarks: 0 particles: 100 min: 1.0 max: 1.0
landmarks: 2 particles: 100 min: 0.14484347866560698 max: 7.049893548954608
landmarks: 1 particles: 100 min: 0.19699310999780262 max: 4.523206706034675
